Processing

# Settings

## Libraries

Loading some python libraries.

In [ ]:
from datetime import timedelta
#import pytz
#import yaml
#import os
#import numpy as np
import pandas as pd
import logging
import pycountry
import json
#from collections import OrderedDict
import sqlite3
import copy
from itertools import chain

# Load raw data

Load the dataset created by the read-script

In [ ]:
data_sets = {}
for resolution in ['15min', '60min']:
    data_sets[resolution] = pd.read_csv(
        'raw_data_' + resolution + '.csv',
        header=[0,1,2,3,4],
        index_col=0, #'timestamp',
        parse_dates=True #'timestamp',
        )

## Creating German data from individual TSOs
The in-feed data for the 4 German controll areas is summed up.

In [ ]:
HEADERS = ['variable', 'country', 'attribute', 'source', 'web']

In [ ]:
web = 'http://data.open-power-system-data.org/datapackage_timeseries'
for tech in ['wind', 'pv']:
    for attribute in ['generation', 'forecast']:
        sum_col = pd.Series()
        for tso in ['50hertz', 'amprion', 'tennet', 'transnetbw']:
            add_col = data_sets['15min'][tech, 'DE' + tso, attribute]
            if len(sum_col) == 0:
                sum_col = add_col
            else:
                sum_col = sum_col + add_col.values
                
        # Create a new MultiIndex
        tuples = [(tech, 'DE', attribute, 'own_calculation', web)]
        columns = pd.MultiIndex.from_tuples(tuples, names=HEADERS)
        sum_col.columns = columns
        data_sets['15min'] = data_sets['15min'].combine_first(sum_col)
        
        # Calculate the profile column
        if attribute == 'generation':
            profile_col = sum_col.values / data_sets['15min'][tech, 'DE', 'capacity']
            tuples = [(tech, 'DE', 'profile', 'own_calculation', web)]
            columns = pd.MultiIndex.from_tuples(tuples, names=HEADERS)
            profile_col.columns = columns
            data_sets['15min'] = data_sets['15min'].combine_first(profile_col)

## Create hourly data from 15' data
Most of the renewables in-feed data comes in 15-minute intervals. We resample it to hourly intervals in order to match the load data from ENTSO-E.

In [ ]:
resampled = data_sets['15min'].resample('H').mean()
data_sets['60min'] = data_sets['60min'].combine_first(resampled)

The data that is available in 15 minute resolution is extracted to be saved separately

# Save csv file to disk
Finally, we write the data to CSV format and save it in the directory of this notebook. Two files are created: one in hourly granularity called "timeseries60.csv" (including all data); and one in quarter-hourly granularity called "timeseries15.csv" (including only data avaiable at this resultion).

# Creating the metadata
In this part, we create the metadata that will document the data output in CSV format. The metadata we be stored in JSON format, which is very much like a python dictionary.
## General metadata
First, we define the general metadata for the timeseries datapackage

In [ ]:
metadata = {
    'name': 'opsd-timeseries',
    'title': 'Time-series data: load, wind and solar, prices',
    'description': 'This dataset contains timeseries data of wind and solar ' +
        'in-feed into the grids of German Transmission System Operators ' +
        'as well as load timeseries for 37 European countries from ENTSO-E.',
    'opsd-jupyter-notebook-url': 'https://github.com/Open-Power-System-Data/' +
        'datapackage_timeseries/blob/master/main.ipynb',
    'version': '2016-03-13',
    'opsd-changes-to-last-version': 'Various things...',
    'keywords': [
        'timeseries','electricity','in-feed','capacity','renewables', 'wind',
        'solar','load','tso','europe','germany'
        ],
    'geographical-scope': 'Europe/Germany',
    'licenses': [{
        'url': 'http://example.com/license/url/here',
        'version': '1.0',
        'name': 'License Name Here',
        'id': 'license-id-from-open'
        }],
    'sources': [],
    'maintainers': [{
        'web': 'http://example.com/',
        'name': 'Jonathan Muehlenpfordt',
        'email': 'muehlenpfordt@neon-energie.de'
        }],
    'resources': [{ # The following is an example of how the file-specific metadata is 
        'path': 'path_to.csv', # structured. The actual metadata is created below
        'format': 'csv',
        'mediatype': 'text/csv',
        'schema': {
            'fields': [{
                'name': 'load_AT_actual',
                'description': 'Consumption in Austria in MW',
                'type': 'number',
                'source': {
                    'name': 'Example',
                    'web': 'http://www.example.com'
                    },
                'opsd-properties': {
                    'Country': 'AT',
                    'Variable': 'load',
                    }
                }]
            }
        }]
    }

indexfield = {
    'name': 'timestamp',
    'description': 'Start of timeperiod in UTC',
    'type': 'datetime',
    'format': 'YYYY-MM-DDThh:mm:ssZ'
    }

descriptions = {
    'net': 'Consumption in {geo} in MW',
    'generation': 'Actual {tech} generation in {geo} in MW',
    'forecast': '{tech} day-ahead generation forecast in {geo} in MW',
    'capacity': '{tech} capacity in {geo} in MW',
    'profile': 'Share of {tech} capacity producing in {geo}',
    'offshoreshare': '{tech} actual offshore generation in {geo} in MW'
    }

## Columns specific metadata
For each dataset/outputfile, the metadata has an entry in the "resources" list that describes the file/dataset. The main part of each entry is the "schema" dictionary, consisting of a list of 'fields", meaning the columns in the dataset. The first field is the timestamp index of the dataset. For the other fields, we iterate over the columns of the MultiIndex index of the datasets to contruct the each field's metadata.

At the same time, a copy of the datasets is created that has a single line column index instead of the MultiIndex.

In [ ]:
data_sets_singleindex = copy.deepcopy(data_sets)##########################
resources = []
for res_key, data_set in data_sets.items():
    columns_singleindex = [] ####################
    fields = [indexfield]
    for col in data_set.columns:
        h = {k: v for k, v in zip(HEADERS, col)}
        if len(h['country']) > 2:
            geo = h['country'] + ' control area'
        elif h['country'] == 'NI':
            geo = 'Northern Ireland'
        elif h['country'] == 'CS':
            geo = 'Serbia and Montenegro'
        else:
            geo = pycountry.countries.get(alpha2=h['country']).name

        field = {}    
        field['description'] = descriptions[h['attribute']].format(
            tech=h['variable'], geo=geo)
        field['type'] = 'number'
        field['source'] = {
            'name': h['source'],
            'web': h['web']
            }
        field['opsd-properties'] = {
            'Country': h['country'],
            'Variable': h['variable'],
            }
        components = [h['variable'], h['country']]
        if not h['variable'] == 'load':
            components.append(h['attribute'])
            field['opsd-properties']['Attribute'] = h['attribute']
        field['name'] = '_'.join(components)
        columns_singleindex.append(field['name'])
        fields.append(field)
        
    resource = {
        'path': 'timeseries' + res_key + '.csv',
        'format': 'csv',
        'mediatype': 'text/csv',
        'schema': {'fields': fields}
        }       
    resources.append(resource)
    data_sets_singleindex[res_key].columns = columns_singleindex ###############
    
metadata['resources'] = resources

In [ ]:
fieldstub = {
    'name': 'same as above',
    'description': '',
    'type': '',
    'format': ''
    }

resources2 = resources
for res_key in ['15min', '60min']:
    more_resources = [{
        'path': 'timeseries' + res_key + '.xlsx',
        'format': 'xlsx',
        'mediatype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        'schema': {'fields': [fieldstub]}
        },
        {
        'path': 'timeseries' + res_key + '_multiindex.xlsx',
        'format': 'xlsx',
        'mediatype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        'schema': {'fields': [fieldstub]}
        },
        {
        'path': 'timeseries' + res_key + '_multiindex.csv',
        'format': 'csv',
        'mediatype': 'text/csv',
        'schema': {'fields': [fieldstub]}
        },
        {
        'path': 'timeseries' + res_key + '_stacked.csv',
        'format': 'csv',
        'mediatype': 'text/csv',
        'schema': {'fields': [fieldstub]}
        }]
    resources2.extend(more_resources)
metadata['resources'] = resources2

Execute this to write the metadata to disk

In [ ]:
datapackage_json = json.dumps(metadata, indent=2, separators=(',', ': '))
with open('datapackage.json', 'w') as f:
    f.write(datapackage_json)

data_sets_singleindex = copy.deepcopy(data_sets)
for res_key, data_set in data_sets.items():
    columns_singleindex = []
    for col in data_set.columns:
        h = {k: v for k, v in zip(COLUMN_HEADERS, col)}
        if h['source'] in ['50Hertz', 'Amprion', 'TenneT', 'TransnetBW']:
            h['country'] = h['country'] + h['source'].lower()            
        col_singleindex = h['variable'] + '_' + h['country'] + '_' + h['attribute']
        columns_singleindex.append(col_singleindex)
    data_sets_singleindex[res_key].columns = columns_singleindex

# Writing the data to disk
We prepare different schemas for the data output. Those are 

In [ ]:
data_sets_multiindex = {}
data_sets_stacked = {}
for res_key in ['15min', '60min']:
    data_sets_multiindex[res_key + '_multiindex'] = data_sets[res_key]
    
    stacked = data_sets[res_key].copy()
    stacked.columns = stacked.columns.droplevel(['source', 'web'])
    stacked = stacked.transpose().stack(dropna=True).to_frame(name='data')
    data_sets_stacked[res_key + '_stacked'] = stacked

## Write to SQL-database
This file format is required for the filtering funtion on the OPSD website

In [ ]:
for res_key, data_set in data_sets_singleindex.items():
    f = 'timeseries' + res_key
    ds = data_set.copy()
    ds.index = ds.index.strftime('%Y-%m-%dT%H:%M:%SZ')
    ds.to_sql(f, sqlite3.connect(f + '.sqlite'),
              if_exists='replace', index_label='timestamp') 

## Write to Excel
Warning: This takes about 15 Minutes to complete.

In [ ]:
for res_key, data_set in chain(data_sets_singleindex.items(),
                               data_sets_multiindex.items()):
    if res_key == '60min_multiindex':
        f = 'timeseries' + res_key
        data_set.to_excel(f+ '.xlsx', float_format='%.2f')

## Write to CSV

In [ ]:
for res_key, data_set in chain(data_sets_singleindex.items(),
                               data_sets_multiindex.items()):
                               #data_sets_stacked.items()):
    f = 'timeseries' + res_key
    data_set.to_csv(f + '.csv', float_format='%.2f',
                    date_format='%Y-%m-%dT%H:%M:%SZ')

# Missing data handling
work in progress

In [ ]:
#for col in data_set.columns:
#    df = col.to_frame()
df = data_sets['15min'].iloc[:,20].to_frame()
df['tag'] = ((df.index >= df.first_valid_index()) &
             (df.index <= df.last_valid_index()) &
             df.isnull().transpose().as_matrix()).transpose()

# make another DF to hold info about each region
regs_isnull = pd.DataFrame()

# first row of consecutive region is a True preceded by a False in tags
regs_isnull['start_idx']  = df.index[df['tag'] & ~ df['tag'].shift(1).fillna(False)]

# last row of consecutive region is a False preceded by a True   
regs_isnull['end_idx']  = df.index[df['tag'] & ~ df['tag'].shift(-1).fillna(False)] 

if df['tag'].any():
    # how long is each region
    regs_isnull['spans'] = regs_isnull['end_idx'] - regs_isnull['start_idx'] + timedelta(minutes=15)
    
    # index of the region with the longest span      
    max_idx = regs_isnull['spans'].argmax()

    # we can get the start and end points of longest region from the original dataframe 
    df.ix[regs_isnull.ix[max_idx][['start_idx', 'end_idx']].values]

In [ ]:
regs_isnull
df.ix[regs_isnull.ix[max_idx][['start_idx', 'end_idx']].values]